In [31]:
import pandas as pd
import numpy as np
from functools import reduce

In [32]:
common_mouse_list_data = pd.read_csv("common_mouse_list.csv")
common_mouse_list_filtered = common_mouse_list_data.loc[:,["ENSMUSG_ID", "mm10"]]

sum_num_of_rows =0
common_mouse_list_filtered_special = common_mouse_list_filtered

list_of_filepaths = ["GSM2699154/GSM2699154_E14_B1.csv", "GSM2699155/GSM2699155_E14_B2.csv",
                     "GSM2699156/GSM2699156_E12_B2.csv", "GSM2699157/GSM2699157_E17_B2.csv",
                     "GSM3140915/GSM3140915_E12_v2.csv", "GSM3140916/GSM3140916_E14_v2.csv",
                     "GSM3140917/GSM3140917_E17_1_v2.csv", "GSM3140918/GSM3140918_E17_2_v2.csv",
                     "GSM3140919/GSM3140919_Fev_Cre_noreporter.csv", "GSM3140920/GSM3140920_Fev_Cre_reporter.csv",
                     "GSM3195456/GSM3195456_10XGenomics_Pancreas_E14-5_1_matrix.csv",
                     "GSM3488509/GSM3488509_10XGenomics_Pancreas_E14-5_2_matrix.csv", "GSM3852752/matrix.csv",
                     "GSM3852753/matrix.csv", "GSM3852754/matrix.csv", "GSM3852755/matrix.csv"]

common_mouse_list_filtered_special.head()

,ENSMUSG_ID,mm10
0,ENSMUSG00000101435,mm10_Gm28772
1,ENSMUSG00000044244,mm10_Il20rb
2,ENSMUSG00000069094,mm10_Pde7a
3,ENSMUSG00000105704,mm10_Gm43055
4,ENSMUSG00000033871,mm10_Ppargc1b


In [ ]:
def update_cols_intersect(num_of_file):
    
    if num_of_file == 0:
        cols_intersect = pd.DataFrame([np.sum(aggregate_cols.iloc[x] >0)
                               for x in range(0, aggregate_cols.shape[0])]).transpose()
        cols_intersect.columns = gene_cols

    else:
        cols_intersect = pd.read_csv("presek_kolona.csv")
        cols_intersect = cols_intersect.drop("Unnamed: 0", axis =1)
        add_cols_intersect = pd.DataFrame([np.sum(aggregate_cols.iloc[i, :] >0)
              for i in range(0, aggregate_cols.shape[0])]).transpose()
        add_cols_intersect.columns = gene_cols
        x = cols_intersect.columns.intersection(add_cols_intersect.columns)
        for cols in set(cols_intersect.columns):
            if cols not in x:
                cols_intersect = cols_intersect.drop(cols, axis = 1)
        for cols in set(add_cols_intersect.columns):
            if cols not in x:
                add_cols_intersect = add_cols_intersect.drop(cols, axis = 1)
        

        cols_intersect = cols_intersect.add(add_cols_intersect)

    cols_intersect.to_csv("presek_kolona.csv")

In [40]:
def prep(filepath, num_of_file):
    sample_data = pd.read_csv(filepath)
    sample_id = filepath[3:10] + '_'
    
    sample_data_filtered = common_mouse_list_filtered.set_index("ENSMUSG_ID").join(sample_data.set_index("Index"),
                                                                                   how = "inner")
            
    gene_cols = sample_data_filtered.index.tolist()
    aggregate_cols = sample_data_filtered.drop("mm10", axis =1).reset_index().drop("index", axis =1)

    update_cols_intersect(num_of_file)
    
    sample_data_filtered.drop("mm10", axis =1, inplace =True)
    sample_data_filtered_transposed = sample_data_filtered.transpose()
    num_rows = sample_data_filtered_transposed.shape[0]
    sample_data_filtered_transposed = sample_data_filtered_transposed.set_index(pd.Series(
        [sample_id + str(x) for x in range(1, num_rows+1)]))
    sample_data_filtered_transposed.to_csv(filepath[:11] + sample_id + "prep_file.csv")
    
    return sample_data_filtered.shape[1]

In [ ]:
sum_num_of_rows = 0
i = 0
for filepath in list_of_filepaths:
    sum_num_of_rows += prep(filepath, i)
    i += 1

In [48]:
def filter_cols(filepath):
    
    prep_file = filepath[:11] + filepath[3:10] + '_prep_file.csv'    
    sample_data = pd.read_csv(prep_file)
    sample_data = sample_data.set_index("Unnamed: 0")
    del sample_data.index.name
    
    for cols in sample_data.columns:
        if cols not in cols_intersect:
            sample_data = sample_data.drop(cols, axis = 1)
    
    for i in sample_data.index:
        if np.sum(sample_data.loc[i, :]) < 1000 or np.sum(sample_data.loc[i, :] > 0) < 500:
            sample_data = sample_data.drop(i, axis =0)
            
    sample_data.to_csv(prep_file[:28] + '2.csv')
    

In [ ]:
percentage = sum_num_of_rows / 100
cols_intersect = pd.read_csv("presek_kolona.csv")
cols_intersect = zip(cols_intersect.columns, cols_intersect.iloc[0])
cols_intersect = set(map(lambda x : x[0], filter(lambda x : x[1] > percentage, cols_intersect)))

In [ ]:
for filepath in list_of_filepaths:
    filter_cols(filepath)

In [81]:
def rename(filepath):
    prep_file = filepath[:11] + filepath[3:10] + '_prep_file2.csv'
    sample = pd.read_csv(prep_file)
    columns = ['id']
    i = 0
    for column in sample.columns:
        if i == 0:
            i += 1
            continue
        columns.append('M' + column[-5:] + common_mouse_list_new.loc[column, 'long'])

    sample.columns = columns
    
    sample = sample.reset_index()
    sample = sample.set_index('id')
    sample = sample.drop('index', axis = 1)
    
    sample.to_csv(prep_file[:28] + '3.csv')

In [ ]:
for filepath in list_of_filepaths:
    rename(filepath)